In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('../beit2')
from datamodules import DATAMODULE_REGISTRY
from models import MODEL_REGISTRY
import torch
from pytorch_lightning import LightningModule
import pytorch_lightning as pl
from rich.progress import track
import matplotlib
import matplotlib.pyplot as plt
from transformers import BertTokenizer
import textwrap
plt.rcParams["axes.axisbelow"] = False
matplotlib.rcParams.update({'font.size': 12})

[2024-09-24 23:39:35,368] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/MasterThesis/src/visualizations/../beit2/modeling_finetune.py:473: UserWarning: Overwriting beit_base_patch16_224 in registry with modeling_finetune.beit_base_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def beit_base_patch16_224(pretrained=False, **kwargs):
/root/MasterThesis/src/visualizations/../beit2/modeling_finetune.py:494: UserWarning: Overwriting beit_base_patch16_384 in registry with modeling_finetune.beit_base_patch16_384. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def beit_base_patch16_384(pretrained=False, **kwargs):
/root/MasterThesis/src/visualizations/../beit2/modeling_finetune.py:510: UserWarning: Overwriting beit_large_patch16_224 in registry with modeling_finetune.beit_large_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def beit_

In [2]:
pl.seed_everything(0)

Seed set to 0


0

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
@torch.no_grad()
def get_cluster_examples(model, dataloader, device):
    cluster_mapping = dict()

    for batch in track(dataloader):
        image = batch['image'].to(device)
        pred = model.quantize_image(image)['encoding_scores'].argmax(dim=-1)
        pred = pred.cpu().tolist()
        for i, cluster in enumerate(pred):
            if cluster not in cluster_mapping:
                cluster_mapping[cluster] = []
            cluster_mapping[cluster].append(batch['image_raw'][i])

    return cluster_mapping

In [6]:
def fit_text(figure, axes, text):
    bbox = axes.get_position()
    fig_width_inch = figure.get_size_inches()[0]
    axes_width_inch = bbox.width * fig_width_inch
    char_width_inch = 0.1
    max_chars_per_line = int(axes_width_inch / char_width_inch)
    wrapped_text = "\n".join(textwrap.wrap(text, width=max_chars_per_line))
    return wrapped_text

In [7]:
def disable_ticks(axes):
    axes.set_xticks([])
    axes.set_yticks([])

In [8]:
def plot_cluster_membership(batch, model, cluster_mapping, device):
    n_cluster_examples = 5
    num_cols = n_cluster_examples+2
    num_rows = batch.shape[0]

    image = batch['image'].to(device)
    pred = model.quantize_image(image)['encoding_scores'].argmax(dim=-1)
    pred = pred.cpu().tolist()

    captions = tokenizer.batch_decode(batch['text'], skip_special_tokens=True)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 2.5*num_rows))

    axes[0, 0].set_title("Caption", fontsize=18, fontweight='bold', fontname='DejaVu Serif')
    axes[0, 1].set_title("Image", fontsize=18, fontweight='bold', fontname='DejaVu Serif')
    for r_idx, j in enumerate(range(2, num_cols), start=1):
        axes[0, j].set_title(f"Member {r_idx}", fontsize=18, fontweight='bold', fontname='DejaVu Serif')

    for i, caption in enumerate(captions):
        axes[i, 0].text(0.5, 0.5, fit_text(caption), ha='center', va='center', fontsize=12)
        axes[i, 0].axis('off')
        axes[i, 1].imshow(batch['image_raw'][i].permute(1, 2, 0))
        disable_ticks(axes[i, 1])
        for spine in axes[i, 1].spines.values():
            spine.set_edgecolor("black")
            spine.set_linewidth(1)

        pred_cluster_idx = pred[i]

        for j, cluster in enumerate(cluster_mapping[pred_cluster_idx][:n_cluster_examples], start=2):
            axes[i, j].imshow(cluster.permute(1, 2, 0))
            disable_ticks(axes[i, j])
            for spine in axes[i, j].spines.values():
                spine.set_edgecolor("black")
                spine.set_linewidth(1)
    
    plt.tight_layout()
    plt.show()

In [10]:
from models.image_vq import ImageVQ, ImageVQLightningModule
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model:ImageVQ = ImageVQLightningModule.load_from_checkpoint("/workspace/models/image_vq_8/model-20016-0.1535-train.ckpt", strict=False).model
model = model.to(device)
model.requires_grad_(False)
model.eval()

In [ ]:
imagenet_args = {
    'data_path': '/workspace',
    'pretraining': True,
    'batch_size':256,
    'num_workers':5,
    'shuffle':False,
    'drop_last':False,
}
imagenet = DATAMODULE_REGISTRY['imagenet'](**imagenet_args)

In [ ]:
imagenet.prepare_data()
imagenet.setup('fit')
dl_imagenet = imagenet.val_dataloader()

In [ ]:
cluster_mapping = get_cluster_examples(model, dl_imagenet, device)

In [ ]:
del imagenet, dl_imagenet

In [11]:
coco_dm_kwargs = {
    'data_path': '/workspace',
    'num_max_bpe_tokens': 64,
    'color_jitter': None,
    'beit_transforms': False,
    'crop_scale': [1.0, 1.0],
    'batch_size': 20,
    'num_workers': 2,
    'shuffle': True,
    'drop_last': False,
}

In [12]:
pl.seed_everything(42)
coco_dm = DATAMODULE_REGISTRY['coco_captions'](**coco_dm_kwargs)

Seed set to 42


In [13]:
coco_dm.prepare_data()
coco_dm.setup('test')

2024-09-24 23:39:38 | INFO | datasets_.base_datasets | [COCOCaptions]: Data already exists under: /workspace/coco
2024-09-24 23:39:38 | INFO | datasets_.base_datasets | [COCOCaptions]: Data already exists under: /workspace/coco
2024-09-24 23:39:39 | INFO | datasets_.base_datasets | [COCOCaptions]: Data already exists under: /workspace/coco
2024-09-24 23:39:39 | INFO | datasets_.base_datasets | [COCOCaptions]: Load 25010 image-text pairs from /workspace/coco/coco_captioning.test.jsonl. 


In [14]:
dl = coco_dm.test_dataloader()
dl_iter = iter(dl)

In [ ]:
batch = next(dl_iter)
plot_cluster_membership(batch, model, cluster_mapping, device)